In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys


import time
import pandas as pd
from urllib.parse import quote
from collections import deque


In [2]:
base_url = "https://www.youtube.com/"
search_query = "deepseek"
search_url = f"{base_url}results?search_query={quote(search_query)}"

print(search_url)

https://www.youtube.com/results?search_query=deepseek


In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url=search_url)
filter_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#filter-button button"))
)

try:
    filter_button_element = driver.find_element(By.CSS_SELECTOR, "#filter-button button")
    if filter_button_element.get_attribute("aria-label") == "Search filters":
        filter_button_element.click()
    else:
        print("No filter button found")
except Exception as e:
    print(f'exception in filter button: {e}')

while True:
    try:
        WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'ytd-search-filter-renderer'))
            )

        duration_element = driver.find_elements(By.TAG_NAME, 'ytd-search-filter-renderer')
        for element in duration_element:
            if element.find_element(By.ID, 'label').get_attribute('title') == "Search for This year":
                element.find_element(By.ID, 'endpoint').click()
                break
        
    except Exception as e:
        print(f'exception in year filter element:')
    break
    

Title = []
Channel = []   
Channel_link = [] 
Views = []
Duration = []
Description = []
Upload_date = []
Video_link = []

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'ytd-video-renderer'))
    )



video_set = set(driver.find_elements(By.TAG_NAME, 'ytd-video-renderer'))
video_elements = deque(video_set)
print(f'video_elements initially: {len(video_set)}')
i = 0


main_window = driver.current_window_handle

while len(video_elements) > 0:
    video_element = video_elements.popleft()

    
    print(f'{i}/, video_elements) {len(video_elements)}')
    i += 1


    try: #dynamic scroll
        
        if len(video_elements) <= 3:
            driver.switch_to.window(main_window)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.TAG_NAME, 'ytd-video-renderer'))
                )

            new_video_set = set(driver.find_elements(By.TAG_NAME, 'ytd-video-renderer'))

            new_video_set -= video_set
            
            video_set = video_set.union(new_video_set)
            video_elements += deque(new_video_set)

            print(f'new video set: {len(new_video_set)}')
            print(f'video_elements total: {len(video_elements)}')
    except Exception as e:
        print(f'exception in dynamic scroll: {e}')

    #thumbnail - duration
    try:
        thumbnail_element = video_element.find_element(By.ID, 'thumbnail')
        duration = thumbnail_element.find_element(By.CLASS_NAME , 'badge-shape-wiz__text').get_attribute("innerText").strip()
        
        if duration == 'SHORTS' or duration.lower == "LIVE NOW".lower or duration.lower == "Premiered".lower or duration.lower == "Upcoming".lower or duration.lower == "SHORTS".lower:
            print(f'\n\nlive now, premiered, upcoming, shorts\n\n')
            continue
        else:
            print('not a shorts, live now, premiered, upcoming')

        print(f'duration : "{duration}"')
    except Exception as e:
        print(f'exception in duration time:')    
        continue



    #channel metadata
    try:

        WebDriverWait(video_element, 10).until(
            EC.presence_of_element_located((By.ID, 'channel-name'))
        )


        channel_element = video_element.find_element(By.ID, 'channel-name')
        channel = channel_element.find_element(By.TAG_NAME, 'a')

        channel_name = channel.get_attribute("innerText").strip()

        channel_link = channel.get_attribute('href')
        
        print(f'channel name: {channel_name}')    
        print(f'channel_link: {channel_link}')
    except Exception as e:
        print(f'exception in channel')
        continue


    #title and video link
    try:
        title_element = video_element.find_element(By.ID, 'video-title')
        title = title_element.get_attribute('title')
        video_link = title_element.get_attribute('href')
        print(f'title: {title}')
        print(f'video_link: {video_link}')
    except Exception as e:
        print(f'exception in title and video link:')
        continue




    #moving to video page
    try:
        print(f'before moving {driver.current_url}')
        driver.execute_script(f"window.open('{video_link}', '_blank')")
        driver.switch_to.window(driver.window_handles[-1])
        print(f'after moving {driver.current_url}')

    except Exception as e:
        print(f'exception in moving to video page:')
        continue




    #expand
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//ytd-text-inline-expander[contains(@id, 'description-inline-expander')]"))
        )
        driver.find_element(By.XPATH, "//ytd-text-inline-expander[contains(@id, 'description-inline-expander')]").click()
        print(f'expand clicked')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    except Exception as e:
        print(f'exception in expand:')
        continue



    # description-element
    try:
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@id='description-inner']"))
        )
        description_element = driver.find_element(By.XPATH, "//div[@id='description-inner']")

        description_span_elements = description_element.find_element(By.ID, 'info-container').find_elements(By.TAG_NAME, 'span')
        views = ""
        upload_date = ""
        if len(description_span_elements) > 2:
            views = description_span_elements[0].get_attribute('innerText').strip()

            upload_date = description_span_elements[2].get_attribute('innerText').strip()
        else:
            print(f'description span elements: {len(description_span_elements)}')
            

        

        description = ' '.join([span.get_attribute('innerText').strip() for span in description_element.find_element(By.XPATH, "//ytd-text-inline-expander[@id='description-inline-expander']").find_elements(By.TAG_NAME, 'span')])
        print(f'description: {description}')

    except Exception as e:
        print(f'exception in description element:')
        continue

        
    Title.append(title)
    Channel.append(channel_name)
    Views.append(views)
    Duration.append(duration)
    Description.append(description)
    Upload_date.append(upload_date)
    Channel_link.append(channel_link)
    Video_link.append(video_link)   

    
















    driver.close()
    driver.switch_to.window(main_window)





    if len(Title) > 25:
        print("{} videos Data collection completed ".format(len(Title)))
        # break


data = pd.DataFrame({
    'Title': Title,
    'Channel': Channel,
    
    'Views': Views,
    'Duration': Duration,
    'Description': Description,
    'Upload_date': Upload_date,
    'Video_link': Video_link,
    'Channel_link': Channel_link,
})


video_elements initially: 20
0/0, video_elements) 19
not a shorts, live now, premiered, upcoming
duration : "20:24"
channel name: 90s Mentor
channel_link: https://www.youtube.com/@90sMentor
title: How to Earn with DeepSeek AI | Create a Tool Website Using Deepseek AI✅ DeepSeek R1
video_link: https://www.youtube.com/watch?v=FftV0cBAUjU&pp=ygUIZGVlcHNlZWs%3D
before moving https://www.youtube.com/results?search_query=deepseek&sp=EgIIBQ%253D%253D
after moving https://www.youtube.com/watch?v=FftV0cBAUjU&pp=ygUIZGVlcHNlZWs%3D
expand clicked
description: Get your Hosting : https://dskills.online/
📢 How to Earn from DeepSeek AI | Create & Monetize a Tool Website! 🚀💰

Want to make money online using DeepSeek AI? 🤖💡 In this video, I’ll show you how to create a free tool website using DeepSeek AI and monetize it for passive income! 🔥💻

🔹 What You’ll Learn:
✅ How to set up a tool website with DeepSeek AI 🛠️
✅ Best monetization strategies for maximum earnings 💰
✅ SEO tips to drive free traffic 🚀
✅ 

In [14]:
len(data)

15

In [18]:

if data['Views'].dtype == 'O':
    data['Views'] = data['Views'].str.replace(r'\D+', '', regex=True).astype(int)

most_viewed_video = data.loc[data['Views'].idxmax()]
print("Most Viewed Video:")
print(most_viewed_video)

if pd.api.types.is_numeric_dtype(data['Duration']):
    avg_duration = data['Duration'].mean()
    print("\nAverage Duration of Videos (in seconds):", avg_duration)
else:
    def duration_to_seconds(duration_str):
        parts = duration_str.split(':')
        if len(parts) == 3:
            hours, minutes, seconds = parts
            return int(hours) * 3600 + int(minutes) * 60 + int(seconds)
        elif len(parts) == 2:
            minutes, seconds = parts
            return int(minutes) * 60 + int(seconds)
        else:
            return 0

    data['Duration_seconds'] = data['Duration'].apply(duration_to_seconds)
    avg_duration = data['Duration_seconds'].mean()
    print("\nAverage Duration of Videos (in seconds):", avg_duration)


Most Viewed Video:
Title               Deepseek | China's New AI Model Destroys Ameri...
Channel                                                  Dhruv Rathee
Views                                                         8883267
Duration                                                        19:39
Description         🤖 Join AI Chatbots Course: https://academy.dhr...
Upload_date                                               Feb 4, 2025
Video_link          https://www.youtube.com/watch?v=FooC7gp4wk4&pp...
Channel_link                     https://www.youtube.com/@dhruvrathee
Duration_seconds                                                 1179
Name: 8, dtype: object

Average Duration of Videos (in seconds): 789.8


In [6]:
data.to_csv('youtube_data.csv', index=False)